<a href="https://colab.research.google.com/github/kevincong95/cs231n-emotiw/blob/master/notebooks/ensemble_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Video Sentiment Analysis in the Wild
### Ensembling Notebook | CS231n

This notebook preprocesses input videos to extract faces, frames, poses, and audio before running pre-trained models for each modality to predict group sentiment (positive, negative, or neutral). 

In [0]:
# Clone the code base
!git clone 'https://github.com/kevincong95/cs231n-emotiw.git'


# Install required packages 
!pip install -r  '/content/cs231n-emotiw/requirements.txt'


Cloning into 'cs231n-emotiw'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 318 (delta 30), reused 51 (delta 22), pack-reused 254
Receiving objects: 100% (318/318), 169.85 MiB | 28.64 MiB/s, done.
Resolving deltas: 100% (163/163), done.
     |████████████████████████████████| 389kB 9.5MB/s 
     |████████████████████████████████| 634kB 17.6MB/s 
     |████████████████████████████████| 3.3MB 30.6MB/s 
     |████████████████████████████████| 26.9MB 113kB/s 
     |████████████████████████████████| 337kB 39.1MB/s 
     |████████████████████████████████| 25.4MB 119kB/s 
     |████████████████████████████████| 100.2MB 40kB/s 
  Created wheel for moviepy: filename=moviepy-1.0.3-cp36-none-any.whl size=110728 sha256=dc406a4071ab8a335d920f756ac6bcb8c5ebe04e1af30ccafdfc208efe90b340
  Stored in directory: /root/.cache/pip/wheels/e0/fe/1c/f4e6dca9e828d4b979c04e461d7fcc5b8e7bd35f947e665b65
  Creat

#### Navigate to the repo we downloaded
We will run all our commands from this repo

In [0]:
!pwd
import os
os.chdir('/content/cs231n-emotiw')
!pwd

/content
/content/cs231n-emotiw


#### Pose Pre-Requisites
Pose extraction uses the [CMU OpenPose library](https://github.com/CMU-Perceptual-Computing-Lab/openpose) to extract body keypoints. We have pre-compiled this library for use in Colab but some system files still need to be installed. 

In [0]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
!wget https://storage.googleapis.com/cs231n-emotiw/openpose/openpose.tar.gz
!tar -xzf openpose.tar.gz

Selecting previously unselected package libgflags2.2.
(Reading database ... 144439 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

#### Set up Google Drive and GCS
(if needed)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Updated property [core/project].
gs://cs231n-emotiw/


#### Retrieve the files

The code block below demonstrates how to retrieve the files from GCS. However, feel free to skip this step if the files are already on the local disk or you have Google Drive mounted.

In [0]:
!wget https://storage.googleapis.com/cs231n-emotiw/data/train-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/val-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/test-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/Train_labels.txt
!wget https://storage.googleapis.com/cs231n-emotiw/data/Val_labels.txt

--2020-05-29 07:34:57--  https://storage.googleapis.com/cs231n-emotiw/data/train-tiny.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.15.128, 2a00:1450:400c:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.15.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50131084 (48M) [application/zip]
Saving to: ‘train-tiny.zip’

train-tiny.zip      100%[===================>]  47.81M  13.2MB/s    in 3.6s    

2020-05-29 07:35:01 (13.2 MB/s) - ‘train-tiny.zip’ saved [50131084/50131084]

--2020-05-29 07:35:03--  https://storage.googleapis.com/cs231n-emotiw/data/val-tiny.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.184.128, 2a00:1450:400c:c0a::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.184.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48020782 (46M) [application/zip]
Saving to: ‘val-tiny.zip’

val-tiny.zip        100%[==============

#### Pre-Processing

Here, we will instantiate each of the preprocessors and process all of the input video files.

NOTE: Change the input parameters as needed.

WARNING: This may take several hours to complete, depending on the number of files.

In general, pre-processing will extract the following:
- Video frames
- Pose keypoints
- Faces from each video frame
- Audio waveform and audio features

In [0]:
from src.preprocessors.scene_preprocessor import VideoPreprocessor
from src.preprocessors.face_preprocessor import FacePreprocessor
from src.preprocessors.pose_preprocessor import PosePreprocessor
from src.preprocessors.audio_preprocessor import AudioPreprocessor

video_preprocessor = VideoPreprocessor(
    video_folder= "train-tiny.zip", 
    label_file= "Train_labels.txt", 
    output_folder="train-tiny-local", 
    output_file= "train-tiny-local.zip"
)

face_preprocessor = FacePreprocessor(
    video_folder="train-tiny.zip",
    output_folder="train-tiny-faces", 
    output_file="train-tiny-faces.zip"
)

pose_preprocessor = PosePreprocessor(
    video_frame_folder="val-tiny.zip",
    output_folder="val-tiny-pose", 
    output_file="val-tiny-pose.zip"
)

audio_preprocessor = AudioPreprocessor(
    output_folder="train-tiny-audio", 
    output_file= "train-tiny-audio.zip" ,
    video_folder= "train-tiny.zip",
    label_path = "Train_labels.txt"
)

preprocessors_list = [video_preprocessor, face_preprocessor, pose_preprocessor, audio_preprocessor] 

for preprocessor in preprocessors_list:
    preprocessor.preprocess()


Using TensorFlow backend.


Video Preprocessor created with is_zip = True, video_folder = train-tiny.zip , label_file = Train_labels.txt , output_folder = train-tiny-local, output_file = train-tiny-local.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Video Preprocessor created with is_zip = True, video_folder = train-tiny.zip , output_folder = train-tiny-faces, output_file = train-tiny-faces.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Pose Preprocessor created with is_zip = True, is_test = False, video_frame_folder = val-tiny.zip , output_folder = val-tiny-pose, output_file = val-tiny-pose.zip
Video Preprocessor created with is_zip = True, video_folder = train-tiny.zip , output_folder = train-tiny-audio, output_file = train-tiny-audio.zip
Frames will be created with hop_size = 0.5
Unzipping files to temp dir train-tiny-local_tmp...
Finished unzipping files
Found 50 videos
Processing video 6/50 with name 122_1.mp4 and class 1 

Processing video 2/5

RuntimeError: ignored

#### Predictions
After performing pre-processing


In [0]:
def predict(mp4_dir , train_target_path , model_list= [model,] , model_paths=["",] , num_models = 1 , mode="soft" , complexFusion=False):

  """

  Notes: 

  All file preprocessing will occur in this function.


  Inputs
  
  * mp4_train_dir - The directory of .mp4 file paths that the model will make predictions from.
  * train_target_path - The target path. Should be in .txt format.
  * A list of pretrained models that will be used in the ensemble during prediction


  Outputs 

  * An array of size (M) where M is the number of samples

  """

  assert len(model_list) == num_models
  assert len(model_paths) == num_models

  all_model_predictions = []
  X_list = []

  counter = 0

  audio = AudioPreprocessor()
  face = 

  # for model in models:
  #    # Do preprocess here on a model basis.
  #   X =  model.preprocess("video path/*.mp4") # X Can either be a np array or None. Can either read from disk, work with raw files, or return None
  #                                             # if using a data generator.
  #   X_list.append(X)                             

  for model in models:

    model.load_model(model_paths[counter])


    X = X_list[counter]
    
   

    all_model_predictions.append(model.predict(dir="video path/*.mp4" , X=X)) # Predict returns an (M , 3) array
    
    counter += 1

  all_model_predictions = np.asarray(all_model_predictions , dtype='float32') # (num_models , M , 3)


  assert mode in ["soft" , "hard"]

  if mode == "soft":

    
    # Take the average of each 

    predictions = np.mean(all_model_predictions , axis=0)

    predictions = np.argmax(predictions , axis = 1)

    return predictions  # (M,) in the domain [0,1,2]

  # TODO: Majority vote

  positive_hard = np.argmax(positive_arr , axis=1) # ()



  return predictions 

In [0]:
def evaluate(predictions, targets):

  # TO DO: Add evaluate code here

  """
  Inputs
  
  * predictions np array of shape M where M is the number of samples
  * target array of shape M where M is the number of samples


  Outputs 

  * Model accuracy scalar 

  """

  assert len(targets) == len(predictions)

  incorrect = np.count_nonzero(predictions - targets)

  acc = (targets.shape[0] - incorrect) / targets.shape[0]

  return acc


